<h1>Data Exploration and Risk Clause Detection</h1>

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

df = pd.read_csv('../data/processed/cleaned_legal_clauses.csv')
df.dropna(subset=['cleaned_text', 'clause_status'], inplace=True)
df.head()

In [2]:
print("Class Distribution:")
print(df['clause_status'].value_counts())

In [3]:
vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
X_tfidf = vectorizer.fit_transform(df['cleaned_text'])
feature_names = vectorizer.get_feature_names_out()

risky_indices = df[df['clause_status'] == 1].index
safe_indices = df[df['clause_status'] == 0].index

risky_tfidf_mean = X_tfidf[risky_indices].mean(axis=0)
safe_tfidf_mean = X_tfidf[safe_indices].mean(axis=0)

risky_mean_array = np.array(risky_tfidf_mean).flatten()
safe_mean_array = np.array(safe_tfidf_mean).flatten()

top_risky_indices = risky_mean_array.argsort()[-20:][::-1]
print("Top words indicating risk in clauses:")
for idx in top_risky_indices:
    print(f"{feature_names[idx]}: {risky_mean_array[idx]:.4f}")

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, df['clause_status'], test_size=0.2, random_state=42)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)

y_pred = rf_model.predict(X_test)
print(classification_report(y_test, y_pred))